In [1]:
import numpy as np
import yaml
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from skopt import gp_minimize
from scipy.optimize import minimize
from scipy.optimize import dual_annealing

## PBD implementation

In [2]:
def generate_edge_constraints(p, e):
    return np.sqrt(((p[e[:, 0]] - p[e[:, 1]])**2).sum(axis=1))

def generate_triangle_constraints(p, t):
    a, b, c = p[t[:, 0]], p[t[:, 1]], p[t[:, 2]]
    return np.cross(b - a, c - a)

def step_start(p, q, v, a, dt):
    v += a * dt
    q[:] = p[:]
    p += v * dt

def collide_with_floor(p, q, mu):
    id = p[:, 1] < 0
    fn = p[:, 1][id]
    p[:, 1][id] = 0

    dx = p[:, 0][id] - q[:, 0][id]
    s = np.sign(dx)
    dx = np.minimum(np.abs(dx), mu * np.abs(fn)) * s
    p[:, 0][id] -= dx

def edge_constraint(p, w, e, l0, k=1.):
    for i in range(len(e)):
        e0, e1 = e[i, 0], e[i, 1]
        p0, p1, w0, w1 = p[e0], p[e1], w[e0], w[e1]
        n = p1 - p0
        l = np.linalg.norm(n)
        if l <= 1e-9:
            continue
        n = -n / l
        lam = -(l - l0[i]) / (w0 + w1)
        dx = k * lam * n
        p[e0] += w0 * dx
        p[e1] -= w1 * dx

def triangle_constraint(p, w, t, a0, k=1.):
    for i in range(len(t)):
        t0, t1, t2 = t[i, 0], t[i, 1], t[i, 2]
        p0, p1, p2, w0, w1, w2 = p[t0], p[t1], p[t2], w[t0], w[t1], w[t2]
        
        p01 = p1 - p0
        p02 = p2 - p0
        p12 = p2 - p1
        lam = w0 * np.dot(p12, p12) + w1 * np.dot(p02, p02) + w2 * np.dot(p01, p01)
        if lam <= 1e-7:
            continue

        lam = k / lam
        a = np.cross(p01, p02)
        if a <= 1e-7:
            continue

        dx = lam * (a - a0[i])
        ccw = np.array([[0, -1], [1, 0]])
        p[t0] += dx * w0 * ccw @ p12
        p[t1] += dx * w1 * ccw @ -p02
        p[t2] += dx * w2 * ccw @ p01

def step_end(p, q, v, dt):
    v[:] = (p[:] - q[:]) / dt

## System implementation

In [3]:
class System:
    def __init__(self):
        self.dt = 1. / 60 / 8
        self.g = np.array([0, -50])
        self.tau = 1.
        self.t = 0.

    def initialize(self, data_path):
        with open(data_path) as f:
            data = yaml.safe_load(f)
            
            self.positions = np.array(data['positions'])
            self.edges = np.array(data['edges'])
            self.triangles = np.array(data['triangles'])
            self.weights = np.array(data['weights'])
            
            self.previous_positions = np.zeros(self.positions.shape)
            self.velocities = np.zeros(self.positions.shape)
            self.constraint_e = generate_edge_constraints(self.positions, self.edges)
            self.constraint_t = generate_triangle_constraints(self.positions, self.triangles)

            self.oscilators = np.zeros((len(self.positions), 3))
            #self.oscilators[0] = np.array([0.2, 25, 0])

    def copy_from(self, other):
        self.dt = other.dt
        self.g = other.g
        self.tau = other.tau
        self.t = other.t

        self.positions = other.positions.copy()
        self.edges = other.edges.copy()
        self.triangles = other.triangles.copy()
        self.weights = other.weights.copy()

        self.previous_positions = other.previous_positions.copy()
        self.velocities = other.velocities.copy()
        self.constraint_e = other.constraint_e.copy()
        self.constraint_t = other.constraint_t.copy()

        self.oscilators = other.oscilators.copy()

    def step(self):
        step_start(self.positions, self.previous_positions, self.velocities, self.g, self.dt)
        gensin = lambda o, e, t: o[e, 0] * np.sin(o[e, 1] * t + o[e, 2])
        factor = lambda o, e, t: gensin(o, e[:, 0], t) + gensin(o, e[:, 1], t)
        ramp = min(1.-(self.tau-self.t) / self.tau, 1)
        f = 1 + ramp * factor(self.oscilators, self.edges, self.t)
        #triangle_constraint(self.positions, self.weights, self.triangles, self.constraint_t, k=.5)
        edge_constraint(self.positions, self.weights, self.edges, f * self.constraint_e, k=.5)
        collide_with_floor(self.positions, self.previous_positions, mu=10)
        step_end(self.positions, self.previous_positions, self.velocities, self.dt)  

    def simulate(self, x=None):
        self.__init__()
        if x is not None:
            x.append(self.positions.copy())
        for _ in range(8 * 600):
            self.step()
            self.t += self.dt
            if x is not None:
                x.append(self.positions.copy())

    def mean_position(self): 
        return self.positions.mean(axis=0)

In [4]:
system_raw = System()
system_raw.initialize('data.yaml')
system = System()

def black_box_function(x):
    system.copy_from(system_raw)
    system.oscilators = np.array(x).reshape(system.oscilators.shape)

    mu0 = system.positions.mean(axis=0)[0]
    system.simulate()
    mu = system.positions.mean(axis=0)[0]
    dist = mu0 - mu

    global i, sol
    sol = max(sol, -dist)
    print(i, -dist, sol)
    i += 1

    return dist

def update(i, x, e):
    ax.clear()
    ax.set_aspect('equal')
    ax.set_xlim((-1, +10))
    ax.set_ylim((-0.1, 3))
    ax.scatter(x[i][:, 0], x[i][:, 1], color='blue')
    for ei in e:
        p = x[i][ei]
        ax.plot(p[:, 0], p[:, 1], color='blue')

## Brute force

In [19]:
np.random.seed(42)
s = system_raw.oscilators.shape

i, sol = 0, 0
best = 0
bf_result = np.random.rand(s[0], s[1])
for _ in range(10):
    #(0, 0.2), (0., 30), (-np.pi, np.pi)
    o = np.random.rand(s[0], s[1])
    o[:, 0] = 0.1 * o[:, 0]
    o[:, 1] = 20 * o[:, 1]
    o[:, 2] = 2 * np.pi * o[:, 2] - np.pi

    if black_box_function(o.flatten()) < best:
        best = sol
        bf_result = o.copy()

0 -0.272057284747778 0
1 0.6945628673366091 0.6945628673366091
2 -0.2241080389194996 0.6945628673366091
3 -0.14009668204046988 0.6945628673366091
4 -2.2935158475978015 0.6945628673366091
5 -0.19955863094338389 0.6945628673366091
6 0.3510254223513163 0.6945628673366091
7 -1.8540408905786951 0.6945628673366091
8 -0.9996792655398048 0.6945628673366091
9 2.0647091565524986 2.0647091565524986


In [12]:
system.copy_from(system_raw)
system.oscilators = bf_result
system.simulate(x := [])

fig, ax = plt.subplots()
ani = FuncAnimation(fig, update, frames=len(x[::16]), interval=100, fargs=(x[::16], system_raw.edges,))
ani.save('ani-opt-brute-force.gif')
plt.close()

## `scipy.optimize`

In [32]:
sol, i = 0, 0
result = minimize(lambda x: black_box_function(x), 
  x0=bf_result.flatten(), 
  method='L-BFGS-B', 
  bounds=[(0, 0.1), (0, 20), (-np.pi, np.pi)] * len(system_raw.oscilators),
  options={ 
    'maxiter' : 2,
    'maxfun' : 5,
    'maxls' : 5,
  }
)

0 2.0647091565524986 2.0647091565524986
1 2.1846167510355876 2.1846167510355876
2 2.2902941837259125 2.2902941837259125
3 1.4424603457263325 2.2902941837259125
4 2.5449210084063107 2.5449210084063107
5 2.270664341480241 2.5449210084063107
6 2.760236972579908 2.760236972579908
7 2.073361895294439 2.760236972579908
8 2.2645851028864667 2.760236972579908
9 3.1262346869436763 3.1262346869436763
10 1.8623666537647288 3.1262346869436763
11 1.7033973655486663 3.1262346869436763
12 2.795341177238006 3.1262346869436763
13 1.652650415601375 3.1262346869436763
14 1.5936748767642932 3.1262346869436763
15 1.8354605564119133 3.1262346869436763
16 2.671323039457138 3.1262346869436763
17 2.040679834230198 3.1262346869436763
18 1.5539065598949873 3.1262346869436763
19 2.1660567230926824 3.1262346869436763
20 1.579079888739093 3.1262346869436763
21 1.5546342139041291 3.1262346869436763
22 2.9569192952685723 3.1262346869436763
23 2.290202428429281 3.1262346869436763
24 1.9314854351265467 3.12623468694367

In [33]:
result

  message: ABNORMAL_TERMINATION_IN_LNSRCH
  success: False
   status: 1
      fun: -1.8929263834593026
        x: [ 9.405e-02  1.908e+01 ...  2.802e+00  1.152e-01]
      nit: 0
      jac: [-1.199e+07 -2.256e+07 ... -2.255e+07  1.332e+07]
     nfev: 150
     njev: 6
 hess_inv: <24x24 LbfgsInvHessProduct with dtype=float64>

In [27]:
system.copy_from(system_raw)
system.oscilators = result.x.reshape(system.oscilators.shape)
mu0 = system.mean_position()[0]
system.simulate()
mu = system.mean_position()[0]
mu - mu0

2.0647091565524986

## Bayesian

In [34]:
system.copy_from(system_raw)
system.oscilators = bf_result
mu0 = system.mean_position()[0]
system.simulate()
mu = system.mean_position()[0]
mu - mu0

2.0647091565524986

In [36]:
sol, i = 0, 0
result = gp_minimize(
    func=lambda x: black_box_function(x), 
    dimensions=[(0, 0.1), (0., 20), (-np.pi, np.pi)] * len(system_raw.oscilators),
    n_calls=1_00,
    x0=list(bf_result.flatten()),
    n_jobs=8)

0 2.0647091565524986 2.0647091565524986
1 -3.3602369290131335 2.0647091565524986
2 -3.9101794345486005 2.0647091565524986
3 -0.40752723230228316 2.0647091565524986
4 -0.5930279332935577 2.0647091565524986
5 0.1653502809900642 2.0647091565524986
6 -0.809422719002229 2.0647091565524986
7 -1.478496996210538 2.0647091565524986
8 -0.8705832841155698 2.0647091565524986
9 -0.376876952830711 2.0647091565524986
10 -1.08779646502999 2.0647091565524986
11 -1.4285659207817347 2.0647091565524986
12 -1.3880339109830269 2.0647091565524986
13 -0.4864917601622376 2.0647091565524986
14 -0.20869321434629695 2.0647091565524986
15 -0.520449803542022 2.0647091565524986
16 -1.531193941802591 2.0647091565524986
17 -2.7834196446533364 2.0647091565524986
18 0.00013895223127713408 2.0647091565524986
19 -0.014620906686100055 2.0647091565524986
20 0.0351652949595318 2.0647091565524986
21 -0.01633808169525386 2.0647091565524986
22 0.04985825718368764 2.0647091565524986
23 -0.36116224689764187 2.0647091565524986
24 

In [38]:
system.copy_from(system_raw)
system.oscilators = np.array(result.x).reshape(system.oscilators.shape)
mu0 = system.mean_position()[0]
system.simulate(x := [])
mu = system.mean_position()[0]
print(mu - mu0)

fig, ax = plt.subplots()
ani = FuncAnimation(fig, update, frames=len(x[::16]), interval=100, fargs=(x[::16], system_raw.edges,))
ani.save('ani-opt-bayesian.gif')
plt.close()

5.9220766701141585


## Dual annealing

In [44]:
sol, i = 0, 0
result = dual_annealing(
    func=lambda x: black_box_function(x), 
    bounds=[(0, 0.1), (0., 20), (-np.pi, np.pi)] * len(system_raw.oscilators),
    maxiter=10,
    maxfun=100,
    x0=bf_result.flatten()
)

0 2.0647091565524986 2.0647091565524986
1 -3.713451408007771 2.0647091565524986
2 -0.48255965801914136 2.0647091565524986
3 0.2938702092879444 2.0647091565524986
4 0.4208381288659173 2.0647091565524986
5 -0.1353891704996959 2.0647091565524986
6 0.4171887688198703 2.0647091565524986
7 1.0751077920535224 2.0647091565524986
8 -0.03925680277788457 2.0647091565524986
9 -3.174332284637674 2.0647091565524986
10 -0.2190194836078958 2.0647091565524986
11 -0.026984920774683285 2.0647091565524986
12 -0.5930657703314203 2.0647091565524986
13 -1.193049652278666 2.0647091565524986
14 -0.8630222254018821 2.0647091565524986
15 0.1297565797355018 2.0647091565524986
16 0.4608072588029226 2.0647091565524986
17 -0.5695355707330758 2.0647091565524986
18 -0.4783402342292211 2.0647091565524986
19 -0.46652436592825475 2.0647091565524986
20 -0.501692820024614 2.0647091565524986
21 -0.010170295647967231 2.0647091565524986
22 -0.03306920191494411 2.0647091565524986
23 -0.6010411830995415 2.0647091565524986
24 -3

In [45]:
result

 message: ['Maximum number of function call reached during local search']
 success: False
  status: 0
     fun: -2.0647091565524986
       x: [ 9.405e-02  1.908e+01 ...  2.802e+00  1.152e-01]
     nit: 0
    nfev: 1474
    njev: 57
    nhev: 0

# Bibliography

[1] Bringing drawings to life: evolving distributed controllers for hand-drawn
soft-bodied robots
Michał Joachimczak1y, Rishemjit Kaur1;2, Reiji Suzuki1, Takaya Arita1